In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from sklearn.ensemble import RandomForestClassifier
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.nn as nn
from torch.autograd import Variable
import torch
#import bcolz
import time
from utils import *
from sklearn.svm import LinearSVC
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

cwd = os.getcwd()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


data_dir = 'Samples/'
import pandas as pd
df = pd.read_csv('train.csv')

#print(df['Target'].unique().max())
#print(df.head())
lab = df['Target'].values
one_hot = MultiLabelBinarizer()
labs = []
for i in lab:
    labs.append( i.replace(" ", ",").split(','))
# One-hot encode data

data = one_hot.fit_transform(labs)
#print(data)
torch.cuda.is_available()
torch.cuda.set_device(1)
torch.backends.cudnn.benchmark=True
torch.backends.cudnn.fastest = True

class CustomDatasetFromImages():
    def __init__(self, path, transform, labels):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Transforms
        self.trans = transform
        self.to_tensor = transforms.ToTensor()
        # Read the csv file
        self.data_info = pd.read_csv('train.csv', header=None)
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.iloc[1:, 0])
        # Second column is the labels
        self.label_arr = labels
        self.path = path
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
       
        single_image_name = self.path + self.image_arr[index]
        
        # Open image
        img_as_img1 = Image.open(single_image_name+'_blue.png')
        img_as_img2 = Image.open(single_image_name+'_yellow.png')
        img_as_img3 = Image.open(single_image_name+'_green.png')

      
        # Transform image to tensor
        img_as_tensor1 = self.to_tensor(img_as_img1)
        img_as_tensor2 = self.to_tensor(img_as_img2)
        img_as_tensor3 = self.to_tensor(img_as_img3)
        image = torch.cat((img_as_tensor1.reshape(1,512,512), img_as_tensor2.reshape(1,512,512)), dim=0)
        image = torch.cat((image, img_as_tensor3.reshape(1,512,512)), dim=0)
        # Get label(class) of the image based on the cropped pandas column
        tf = transforms.Normalize((0.5,),(1.0,))
        image = tf(image)
        single_image_label = self.label_arr[index]

        return (image, single_image_label)

    def __len__(self):
        return self.data_len
transformer = transforms.Compose([
transforms.RandomVerticalFlip(p=1),
transforms.ToTensor(),
                             transforms.Normalize((0.5,),(1.0,)),

                             ])
    
train = CustomDatasetFromImages(cwd + '/Train/',transform=transformer,labels=data)
train_loader = torch.utils.data.DataLoader(train,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=0)   

use_gpu = torch.cuda.is_available()


device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model_conv = torchvision.models.resnet50(pretrained=True)

#model_conv = models.alexnet(pretrained=True)
#model_conv = models.squeezenet1_0(pretrained=True)
#model_conv = models.vgg16(pretrained=True)
#model_conv = models.densenet161(pretrained=True)
#model_conv = models.inception_v3(pretrained=True)

for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features


class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
    
model_conv.fc = Identity()
#print(model)

model= model_conv.cuda()




optimizer = optim.Adam(model.parameters(), lr=0.00001)

criterion = nn.CrossEntropyLoss()

def feat_extr(loader):
    out =  Variable(torch.FloatTensor())
    labels =  Variable(torch.FloatTensor())
    image = 0
    count = 0
    batch = 0
    for batch_idx, (x, target) in enumerate(train_loader):
        if (batch_idx + 1) * 16 >= 31072:
            break
        
        target = target.float().cuda()
        x = x.cuda()
        
        if len(out.size()) == 1:
            out = model(x).cpu()
            labels = target.cpu()
        else:
            out = torch.cat((out, model(x).cpu()), dim=0)
            labels = torch.cat((labels, target.cpu()), dim=0)
        
    print(out.shape)
    return out, labels   
out, labels = feat_extr(train_loader)
train = out[:24848]
train_labels = labels[:24848]

test = out[24848:]
test_labels = labels[24848:]

print('Features Extracted')
correct_cnt, ave_loss = 0, 0
total_cnt = 0



torch.Size([31056, 204800])
Features Extracted


In [33]:

clf = RandomForestClassifier(n_estimators=128, max_depth=3,
                             random_state=0)
clf.fit(train, train_labels)
#print(train_labels.shape)

preds = []
for i in test:
    preds.append(clf.predict(i.reshape(1,-1)).reshape(-1))



In [34]:
count = 0
TP = 0
FN = 0
FP = 0

for indx in range(test_labels.shape[0]):
    for label in range(test_labels[indx,:].shape[0]):
            
            if test_labels[indx,label].item() > 0.5:
                if preds[indx][label] > 0.5:
                    TP += 1
                else:
                    FN += 1
            else:
                if preds[indx][label] > 0.5:
                    FP += 1

            count += 1
                    
if TP == 0 and FP == 0:
    FP = 1
if TP == 0 and FN == 0:
    FN = 1
precision = TP / (TP + FP)
recall = TP / (TP + FN)
if precision == 0 and recall == 0:
    recall = 1
f1 = 2*((precision*recall)/(precision+recall))

print('Accuracy')
print(f1)

Accuracy
0.21302175937685208
